To Do:


Add bellyflop functionality

Add bounce boxes (metal/wood, one class that adds a shape of an arrow, and that's it, rest will be handled outside)

Add Tnt destruct sequence

Add destruct animations

Add protection box (and following sprite?)

Level saving

Game structure, saving, scores, etc.


Credits:
“Sound effects obtained from zapsplat.com and freesound.org"
"Idea shamelessly taken from Crash Bandicoot 2: Wrath of Cortex for PS2"


In [ ]:
import pygame

pygame.init()

import math
import Box
import numpy as np
from Super_Classes import Vector, Body, Shape
import Platform
from random import randint
import copy
from Player import Player
from Constants import G, box_size, S
from time import sleep
from Make_Sounds import wood_bounce_sound, wood_break_sound, boom_sound


display_size = (1000,600)

gameDisplay = pygame.display.set_mode(display_size)
pygame.display.set_caption('Game!')

black = (0,0,0)
white = (255,255,255)
floor = 500

clock = pygame.time.Clock()

bounce_vel = 6.0*S

R = box_size*4
N = 600
path = [np.array([500*S + R*math.cos(2*math.pi*x/N),floor + R*math.sin(2*math.pi*x/N)]) for x in range(N)]
d1 = Platform.Moving_Platform([box_size*2,box_size/3],copy.copy(path),color=(50,50,50),line_color = None ,line_width = 2)
d2 = Platform.Moving_Platform([box_size*2,box_size/3],path[int(N/2):]+path[:int(N/2)],color=(50,50,50),line_color = (0,0,0) ,line_width = 2)

p0 = Platform.Platform(*[[500*S,floor],[10*S,10*S]])
p0.solid = False
p0.corporeal = False
p1 = Platform.Platform([100*S,floor+50*S],[200*S,50*S])
p2 = Platform.Platform([1000*S,floor+50*S],[300*S,50*S])


a = Box.Nitro(p1.pos - [-box_size*100/30,p1.size[1]+box_size])
a1 = Box.Metal(a.pos - [0,box_size*2])
a1.is_on(a)
a3 = Box.Metal_Wood(a.pos - [box_size*2,box_size*2])
a3.is_off()
a2 = Box.Nitro(a.pos - [box_size*2,0])
a2.is_off()

b2 = Box.Tnt(path[int(N/2)]-[0,box_size+d1.size[1]])
b2.is_on(d2)
b3 = Box.Bouncey_Wood(path[int(N/2)]-[0,box_size*3+d1.size[1]])
b3.is_on(d2)
b4 = Box.Wood(path[int(N/2)]-[0,box_size*5+d1.size[1]])
b4.is_on(d2)
#b1 = Box.Nitro(path[0]-[0,a.size[1]+d2.size[1]])
#b1.is_on(d1)
#b4 = Box.Nitro(path[0]-[0,a.size[1]+d2.size[1]+a.size[1]*2])
#b4.is_on(b1)


master_body_list = [p1,p2,a,a1,a2,a3,d1,d2,b2,b3,b4]
background_list = [p0]
foreground_list = [] # for exploding things, plus lives, score, etc

for k in range(4):
    for i in range(3):
        master_body_list.append(Box.Wood(a.pos + [600*S+box_size*2*k,-i*2*box_size]))
for i in [0,3.0]:
    master_body_list.append(Box.Bouncey_Wood(a.pos + [600*S+box_size*2*6,-i*2*box_size]))
        
ticker = -1

shifts = [np.array([randint(-10,10)/5,randint(-10,10)/5]) for _ in range(5)]
shifts.append(np.array([0,0]))


crashed = False     
            
left_key = False
right_key = False
up_key = False
down_key = False
crouch_key = False
jump_hold = False

relevant_one = None
character= Player([200*S,floor-300*S])

# Object to determine what to draw on screen
screen = Body([0,0],display_size,corporeal=True,solid=False,velocity=[0,0])   
    
death_delay = 60 # time after death level keeps tickin'
crashed = death_delay
body_list = copy.deepcopy(master_body_list)
jump_timer = 0

while True:
    
    if crashed:
        
        if crashed == death_delay:
            character= Player([200*S,floor-300*S])
            crashed = False
            body_list = copy.deepcopy(master_body_list)
            relevant_one = None
        else:    
            crashed += 1

    jump_timer -=1
    jump_key = False
    attack_key = False
    flop_key = False
    
    for event in pygame.event.get(): ### THIS IS NEEDED OR IT WILL CRASH

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                left_key = True
            elif event.key == pygame.K_RIGHT:
                right_key = True
                
            elif event.key == pygame.K_UP:
                up_key = True
            elif event.key == pygame.K_DOWN:
                down_key = True
                
            elif event.key == pygame.K_c:
                crouch_key = True
                flop_key = True
        
            elif event.key == pygame.K_x:
                jump_key = True
                jump_timer = character.jump_anticipation
                jump_hold = True
            
            elif event.key == pygame.K_z:
                attack_key = True
   
            elif event.key == pygame.K_q:
                crashed = True
                
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                left_key = False
            elif event.key == pygame.K_RIGHT:
                right_key = False
            elif event.key == pygame.K_c:
                crouch_key = False
            
            elif event.key == pygame.K_UP:
                up_key = False
            elif event.key == pygame.K_DOWN:
                down_key = False
            elif event.key == pygame.K_x:
                jump_hold = False

        ######################
    
    

         

    gameDisplay.fill(white)

    
    # Move every object including the player
    for bod in body_list:
        bod.move()
    
    if not crashed:
        # determine character's state and velocity
        character.evolve(right_key-left_key,crouch_key,jump_key,attack_key,flop_key,jump_hold)

        character.move()
        squeeze = [[False,False],[False,False]]

        # sort body_list so that closest bodies interact first. 
        # this prevents incorrect interactions like diagonally breaking boxes
        body_list = sorted(body_list, key= lambda x: sum((character.pos-x.pos)**2), reverse=False)



        for k,i in enumerate(body_list):

            if i.solid: # if corporeal object (not fruit)
                dim,gap = character.overlap_dim(i)
                if isinstance(character.hit_box(),Body):
                    hit = character.hit_box().overlap(i)
                else:
                    hit = False

                if dim>-1: # overlap!
                   # print(dim)
                    side = 2*(character.pos[dim] > i.pos[dim]) - 1
                    converging = character.vel[dim]*side < 0 # if character is moving towards object


                    if isinstance(i,Box.Wood):
                        bounce = (dim == 1) and converging and (not hit)
                        if bounce:
                            i.bounces -=1
                        break_box = (bounce and i.bounces == 0) or hit
                        if bounce and not break_box:
                            wood_bounce_sound()

                        if break_box:
                            wood_break_sound()
                            
                    elif isinstance(i,Box.Metal_Wood):
                        bounce = False
                        break_box = character.flopping and hit
                        if break_box:
                            wood_break_sound()

                    elif isinstance(i,Box.Nitro):
                        break_box = True
                        bounce = False
                    elif isinstance(i,Box.Tnt):
                        break_box = hit
                        bounce = dim == 1 and converging and (not hit) # and not timer already started
                        # countdown_timer_start = (dim == 1 and converging) 
                    else:
                        break_box,bounce = False,False


                    if not break_box:  # record squeeze, remove overlap, reduce velocity in this dimension to 0
                        squeeze[dim][(character.pos[dim] > i.pos[dim])] = True
                        character.pos[dim] += side*gap
                        if converging: 
                            character.vel[dim] = 0.0
                    else:
                        i.destroy()
                        if isinstance(i,Box.Nitro) or isinstance(i,Box.Tnt):
                            crashed = True
                            body_list.remove(i)
                            foreground_list.append(i)
                            boom_sound()
                      

                    if bounce: 
                        if not (character.flopping>0) or not break_box: # if character is flopping and box breaks, no bounce
                            character.bounce(jump_timer*jump_hold,side) 
                            # first input is jump was recently pressed and still held


                    if side == -1 and dim == 1 and not break_box and not bounce: 
                        character.is_on(i) # character above, colliding vertically, i is not broken, character not bouncing
                        relevant_one = i

                elif hit: # if hit box overlap but no character overlap
                    if isinstance(i,Box.Wood):
                        i.destroy()
                        wood_break_sound()
                    elif isinstance(i,Box.Metal_Wood):
                        if character.flopping>0:
                            i.destroy()
                            wood_break_sound()
                    elif isinstance(i,Box.Tnt) or isinstance(i,Box.Nitro):
                        i.destroy()
                        crashed = True
                        body_list.remove(i)
                        foreground_list.append(i)
                        boom_sound()
                        
                else:
                    if character.resting_on == i:
                        character.vel += i.vel
                        character.is_off() # not on anything no more
                    if character in i.on_me:
                        i.on_me.remove(character)

            else: #non-corporeal, do not adjust velocity
                pass
               # if isinstance(i,Fruit):
               #     if character.overlap(i):
                #       #i.color = tuple([i*.98 for i in list(i.color)])
                 #       removal.append(i)
                  #      i.visible = False

        # overlaps on both sides for a given dimension, this should by design happen rarely
        if any(all(i) for i in squeeze):
            crashed = True # dead!

        # if character completely off screen, ded.
        if not screen.overlap(character):
            crashed = True #dead!


    # Earthquake maker!
    if ticker == -1 and False:
        ticker = len(shifts)-2  
    elif ticker>=0:
        ticker -=1
    
    
    gameDisplay.fill(white)
    screen.pos = np.array([character.pos[0]-display_size[0]/2,0])
    
   
    big_list = background_list+body_list
    if not crashed:
        big_list += [character]
        
    big_list += foreground_list
    
    for bod in big_list:
        
        if ticker>=0:
            bod.visual_shift(shifts[ticker]-shifts[ticker+1])
        if screen.overlap(bod):
            bod.draw(gameDisplay,screen.pos)

  #  sleep(0.1)
    pygame.display.flip()
    

    
pygame.quit()

/Users/Sam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:230: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


In [1]:
x = 2
master_body_list[x].on_me,master_body_list[x]

NameError: name 'master_body_list' is not defined

In [ ]:
character.resting_on.resting_on

In [ ]:
    squeeze = [[],[]]

In [5]:
character.flopping

0

In [ ]:
type(True)